# Фибоначчи

### Идея:
\begin{gather}
 \begin{pmatrix}
  F_{n+1} & F_n \\
  F_n & F_{n-1}
 \end{pmatrix}
 =
  \begin{pmatrix}
   1 & 1 \\
   1 & 0 \\
   \end{pmatrix}^{\!n}
\end{gather}

Обозначим:
\begin{gather}
 Q
 =
  \begin{pmatrix}
   1 & 1 \\
   1 & 0 \\
   \end{pmatrix}
\end{gather}

Из тождества получаем, что $F_n = Q^{n} (1, 2)$ или $F_n = Q^{n-1} (1, 1)$

Вычисление $F_n$ сводиться к возведению матрицы в степень.
Пусть у нас имеется некоторая матрица $M$, которую необходимо возвести в степень $n\in \mathbb{N}_1$. Предположим также, что $n$ является степенью двойки, т.е. $n=2^i,i\in \mathbb{N}_0$.  
Тогда $M^n$ можно представить в виде дерева:  
![image.png](https://habrastorage.org/storage2/9b8/7a2/d22/9b87a2d22757b3829364031ef890b61c.png)  
Имеется в виду:
$M^n=M^{n/2}\cdot M^{n/2}=...=\prod ^{n}_{1}M^1$.
Соответственно, для вычисления матрицы $M^n$ нужно вычислить матрицу $M^{n/2}$ и умножить саму на себя. Для вычисления $M^{n/2}$ нужно тоже самое проделать с $M^{n/4}$ и т.д.

Теперь же встает вопрос: что делать, если $n$ не является степенью двойки? Любое натуральное число $n$ можно разложить как сумму чисел, являющихся степенью двойки, причем без повторений (мы занимаемся этим каждый раз, когда переводим число из двоичной системы счисления в десятичную).

## Алгоритм

In [ ]:
class Fibonacci:
    Q = [[1, 1], [1, 0]]

    def __init__(self):
        self.cache = {}

    def get_matrix_power(self, M, p):
        if p == 1:
            return M
        if p in self.cache:
            return self.cache[p]
        K = self.get_matrix_power(M, int(p/2))
        R = self.multiply_matrices(K, K)
        self.cache[p] = R
        return R
    
    def multiply_matrices(self, M1, M2):
        a11 = M1[0][0]*M2[0][0] + M1[0][1]*M2[1][0]
        a12 = M1[0][0]*M2[0][1] + M1[0][1]*M2[1][1]
        a21 = M1[1][0]*M2[0][0] + M1[1][1]*M2[1][0]
        a22 = M1[1][0]*M2[0][1] + M1[1][1]*M2[1][1]
        r = [[a11, a12], [a21, a22]]
        return r

    def get_number(self, n):
        if n == 0:
            return 0
        if n == 1:
            return 1
        powers = []
        p = 0
        for digit in reversed(bin(n-1)[2:]):
            if digit == '1':
                powers.append(int(pow(2, p)))
            p += 1
        matrices = [self.get_matrix_power(Fibonacci.Q, p) for p in powers]
        while len(matrices) > 1:
            M1 = matrices.pop()
            M2 = matrices.pop()
            R = self.multiply_matrices(M1, M2)
            matrices.append(R)
        return matrices[0][0][0]
    
    def get_remainder(self, n, m):
        number = self.get_number(n)
        return number % m

In [79]:
fibonacci = Fibonacci()

In [80]:
%%time
fibonacci.get_number(999)

CPU times: user 63 µs, sys: 11 µs, total: 74 µs
Wall time: 76.8 µs


26863810024485359386146727202142923967616609318986952340123175997617981700247881689338369654483356564191827856161443356312976673642210350324634850410377680367334151172899169723197082763985615764450078474174626

## Оптимизированный алгоритм

\begin{gather}
 \begin{pmatrix}
  a & b \\
  b & c
 \end{pmatrix}
 *
 \begin{pmatrix}
  a & b \\
  b & c \\
 \end{pmatrix}
 =
 \begin{pmatrix}
  a^2 + b^2 & ab + bc \\
  ab + bc & b^2 + c^2 \\
 \end{pmatrix}
\end{gather}

In [82]:
def fibonacci(n):
    a, b, c = 1, 1, 0
    for bit in bin(n)[3:]:
        temp = b * b
        a, b, c = a*a + temp, (a + c)*b, temp + c*c
        if bit=='1':
          a, b, c = a + b, a, b
    return b

In [83]:
%%time
fibonacci(999)

CPU times: user 19 µs, sys: 3 µs, total: 22 µs
Wall time: 27.2 µs


26863810024485359386146727202142923967616609318986952340123175997617981700247881689338369654483356564191827856161443356312976673642210350324634850410377680367334151172899169723197082763985615764450078474174626

## Алгоритм с нахождением остатка


In [84]:
def fibonacci_with_remainder(n, m):
    a, b, c = 1, 1, 0
    for bit in bin(n)[3:]:
        temp = (b * b) % m
        a, b, c = (a*a + temp) % m, ((a + c)*b) % m, (temp + c*c) % m
        if bit=='1':
          a, b, c = (a + b) % m, a, b
    return b

In [85]:
%%time
fibonacci_with_remainder(1000000000001, 99999)

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 33.9 µs


63715